In [1]:
import duckdb
from pathlib import Path
import requests
import numpy as np


# Hamming Space search

This is a demonstration 

# Step 1: Load Data

First, we create a local database using duckdb. This isn't strictly necessary: if you want to save some disk space, you can change `CREATE TABLE` below to `CREATE VIEW`, which will simply scan the parquet files in place rather than fill them into a new database table.

In [ ]:
con = duckdb.connect("database.db")
con.execute("CREATE TABLE hamming AS SELECT * FROM parquet_scan('parquet/*.parquet')")

# Step two: set up a hashing instance.

Now we need to build a hasher using SRP. You should be able to install the SRP package using `pip install pysrp`. The SRP hasher does basic whitespace tokenization, which works pretty well for languages with spaces (i.e. not Chinese, Thai, and a few others) and turns text into a vector.

In [9]:
import SRP
hasher = SRP.SRP(1280)

# Step 3 

Get some text. 

Here I grab an arbitrary e-text from Project Gutenberg. Anything should work here--load from your disk, whatever. The only rules are:

1. It should be unicode
2. It should be kind of long: at least a couple hundred words, and ideally a hundred thousand words.

In [78]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers

text = strip_headers(load_etext(271)).strip()

# Step 4: create a vectorized text.

Now we need to use SRP to hash this. I do a couple numpy tricks here: first I do a boolean test on each dimension to test if it's over zero; 
then I take a view of that as a numpy 64-bit integer. That number is only every going to be used as a string of bytes, though.

In [84]:


rep_floating = hasher.stable_transform(text, unit_length=True)
packed = np.packbits(rep_floating > 0).view(np.int64)



# Step 5: Turn the bytes into a single-row database table.

This is pretty straightforward: we just clone the basic database structure, and then insert a new row.

Note that you could have as many elements as you wanted in this query table; you could even do fancy weighting where a text
should be close to some documents and far from others.

In [89]:

con.execute("DROP TABLE IF EXISTS query")
con.execute(f"CREATE TABLE query AS SELECT * FROM hamming LIMIT 1")
con.execute(f"DELETE FROM query")

q = f'''INSERT INTO query VALUES ('NA', {", ".join([str(p) for p in packed])})'''
con.execute(q)

# Step 6: Create a list of searches.

This is a little complicated, because we are doing the Hamming distance on each of the twenty integers: I write a function here to create the bit that will test how far about the query is from the searched texts.

In [91]:


def dist_clause():
  """
  return text of the formula to calculate hamming distance across the 20 chunks of data I provide in the files.
  """
  funcs = []
  for i in range(20):
    letter = chr(i + 65)
    func = f"BIT_COUNT(xor(l.{letter}, r.{letter}))::FLOAT"
    funcs.append(func)
  return f" {' + '.join(funcs)}"

dist_clause()

' BIT_COUNT(xor(l.A, r.A))::FLOAT + BIT_COUNT(xor(l.B, r.B))::FLOAT + BIT_COUNT(xor(l.C, r.C))::FLOAT + BIT_COUNT(xor(l.D, r.D))::FLOAT + BIT_COUNT(xor(l.E, r.E))::FLOAT + BIT_COUNT(xor(l.F, r.F))::FLOAT + BIT_COUNT(xor(l.G, r.G))::FLOAT + BIT_COUNT(xor(l.H, r.H))::FLOAT + BIT_COUNT(xor(l.I, r.I))::FLOAT + BIT_COUNT(xor(l.J, r.J))::FLOAT + BIT_COUNT(xor(l.K, r.K))::FLOAT + BIT_COUNT(xor(l.L, r.L))::FLOAT + BIT_COUNT(xor(l.M, r.M))::FLOAT + BIT_COUNT(xor(l.N, r.N))::FLOAT + BIT_COUNT(xor(l.O, r.O))::FLOAT + BIT_COUNT(xor(l.P, r.P))::FLOAT + BIT_COUNT(xor(l.Q, r.Q))::FLOAT + BIT_COUNT(xor(l.R, r.R))::FLOAT + BIT_COUNT(xor(l.S, r.S))::FLOAT + BIT_COUNT(xor(l.T, r.T))::FLOAT'

Then we can actually run the query, finding the fifty closest books to our seed text. This takes about 3 seconds to scan across 20 million books on my laptop. Not terrible!

In [98]:
q = con.query(f'''
  SELECT SUM({dist_clause()}) AS distance , l.htid AS htid FROM hamming AS l CROSS JOIN query AS r GROUP BY l.htid ORDER BY distance  ASC LIMIT 50
''')

entries = q.arrow()['htid']


You can see what these ids are by checking them in the hathi trust bibliographic API.

In [99]:
for entry in entries[:20]:
  title = [*requests.get(f"https://catalog.hathitrust.org/api/volumes/brief/htid/{entry}.json").json()['records'].items()][0][1]['titles'][0]
  print(f"{title} ({entry})")


Black Beauty; the autobiography of a horse (njp.32101025286947)
Black Beauty (nc01.ark:/13960/t8nc72h97)
Black Beauty : his groom and companions. The "Uncle Tom's cabin" of the horse (hvd.hwp3ii)
Black Beauty : the autobiography of a horse (mdp.39015038609189)
Black Beauty : the autobiography of a horse (wu.89092951672)
Black Beauty, the autobiography of a horse (mdp.39015011405985)
Black Beauty :  the autobiography of a horse (pst.000026473765)
Black Beauty (uva.x001134706)
Black beauty (nc01.ark:/13960/t3jw9gh8b)
Black Beauty, his groom and companions. : The "Uncle Tom's cabin" of the horse (mdp.39015040765490)
Black Beauty (mdp.39076000504394)
Black Beauty : the autobiography of a horse (mmet.ark:/13960/t8z89k129)
Black beauty (uc2.ark:/13960/t2b853x3w)
Black Beauty : the autobiography of a horse (mdp.39076002790348)
Black Beauty; the autobiography of a horse (hvd.hwp3ij)
Black Beauty : his grooms and companions ; the autobiography of a horse (uc2.ark:/13960/t4qj79244)
Black Beauty 

What you do with these matches is up to you: I recommend the [HTRC Feature Reader Library](https://github.com/htrc/htrc-feature-reader).

In [109]:
from htrc_features import Volume
closest = Volume(entries.to_pylist()[0])

In [111]:
closest

In [112]:
closest.tokenlist().sample(10)

,,,,count
page,section,token,pos,
109,body,At,IN,1
176,body,on,IN,1
211,body,I,CD,1
124,body,fits,NNS,1
69,body,the,DT,1
242,body,``,``,3
267,body,health,NN,1
39,body,bleed,VB,1
155,body,you,PRP,2
